In [3]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import scraperwiki
import urllib2
import urllib
import time

In [4]:
url = 'http://www.fightmetric.com/statistics/fighters'
html = scraperwiki.scrape(url)
soup = BeautifulSoup(html, 'lxml')

In [5]:
# page = urllib2.urlopen(url)
# soup = BeautifulSoup(page.read(), 'lxml')
# print soup
# tables = soup.find_all('div', {'class': 'ranking-list tall'})
# tables.extend(soup.find_all('div', {'class': 'ranking-list tall center-row'}))

In [9]:
table_fighter = soup.find_all('tbody')[0]
rows_fighter = table_fighter.find_all('tr')[1:]
# for i in rows_fighter[0]:
#     print i

11

In [49]:
def get_career_stats(url):
    html = scraperwiki.scrape(url)
    soup = BeautifulSoup(html, 'lxml')
    div_stats = soup.find_all('div', {'class': 'b-list__info-box-left clearfix'})
    print div_stats

In [50]:
test_url = 'http://www.fightmetric.com/fighter-details/93fe7332d16c6ad9'
get_career_stats(test_url)


[<div class="b-list__info-box-left clearfix">\n<div class="b-list__info-box-left">\n<i class="b-list__box-item-title">\n        Career statistics:\n      </i>\n<ul class="b-list__box-list b-list__box-list_margin-top">\n<li class="b-list__box-list-item b-list__box-list-item_type_block">\n<i class="b-list__box-item-title b-list__box-item-title_font_lowercase b-list__box-item-title_type_width">\n            SLpM:\n          </i>\n\n          0.00\n\n        </li>\n<li class="b-list__box-list-item b-list__box-list-item_type_block">\n<i class="b-list__box-item-title b-list__box-item-title_font_lowercase b-list__box-item-title_type_width">\n            Str. Acc.:\n          </i>\n          0%\n        </li>\n<li class="b-list__box-list-item b-list__box-list-item_type_block">\n<i class="b-list__box-item-title b-list__box-item-title_font_lowercase b-list__box-item-title_type_width">\n            SApM:\n          </i>\n          0.00\n        </li>\n<li class="b-list__box-list-item b-list__box-

In [41]:
#loop through a single fighter row and capture all the information
def parse_row(row):
    cols_fighter = row.find_all('td')
#     len(cols_fighter) #11 is correct
    for idx, col in enumerate(cols_fighter):
        links = col.find_all('a')
        num_links = len(links)
        if num_links == 1:
            link = links[0]
            try:
                if idx == 0:
#                 print link['href']
                print link.contents[0]
            except IndexError as e:
                print 'n/a'
        elif num_links > 1:
            raise Exception('invalid link parsing')
        else:
    #         print repr(col.contents[0])
            txt = col.contents[0].strip()
            if txt != '':
                if '-' in txt:
                    print 'n/a'
                else:
                    print col.contents[0].strip()
            else:
                print 'n/a'

In [42]:
for row in rows_fighter:
    parse_row(row)

http://www.fightmetric.com/fighter-details/93fe7332d16c6ad9
Tom
http://www.fightmetric.com/fighter-details/93fe7332d16c6ad9
Aaron
http://www.fightmetric.com/fighter-details/93fe7332d16c6ad9
n/a
n/a
155 lbs.
n/a
n/a
5
3
0
n/a
http://www.fightmetric.com/fighter-details/15df64c02b6b0fde
Danny
http://www.fightmetric.com/fighter-details/15df64c02b6b0fde
Abbadi
http://www.fightmetric.com/fighter-details/15df64c02b6b0fde
The Assassin
5' 11"
155 lbs.
n/a
Orthodox
4
6
0
n/a
http://www.fightmetric.com/fighter-details/b361180739bed4b0
David
http://www.fightmetric.com/fighter-details/b361180739bed4b0
Abbott
http://www.fightmetric.com/fighter-details/b361180739bed4b0
Tank
6' 0"
265 lbs.
n/a
Switch
10
14
0
n/a
http://www.fightmetric.com/fighter-details/2f5cbecbbe18bac4
Shamil
http://www.fightmetric.com/fighter-details/2f5cbecbbe18bac4
Abdurakhimov
http://www.fightmetric.com/fighter-details/2f5cbecbbe18bac4
Abrek
6' 3"
235 lbs.
76.0"
Orthodox
16
4
0
n/a
http://www.fightmetric.com/fighter-details/c0ed

In [ ]:
fighters = []
for table in tables:
    wcname = str(table.find('div', {'class': 'weight-class-name'}).contents[0]).strip()
    if 'Pound-for-Pound' in wcname:
        continue
    else:
#         print table.a.contents[0]
        fighters.append(table.a.contents[0])
        other_ranked = table.find_all('td', {'class': 'name-column'})
        for ranked in other_ranked:
            print ranked.a.contents[0]
        

In [ ]:
fighters

In [ ]:
names = []
for table in tables:
    all_td = table.find_all('a')
    all_td_str = set([x.string for x in all_td if x.string is not None])
    all_td_str = [x.encode('utf-8').strip() for x in all_td_str if 'County' not in x]
    all_td_str = [x for x in all_td_str if 'District' not in x and 'spade' not in x and 'Piercer' not in x and '\xbc' not in x and 'Generals' not in x and 'Marquis' not in x]
    all_td_str = [x for x in all_td_str if 'home' not in x and 'Yama' not in x]
    all_td_str = [x for x in all_td_str if 'Spade' not in x]
    all_td_str = [x for x in all_td_str if ' ' in x]
    all_td_str = [[x] for x in all_td_str if len(x.split(' ')) < 4]
    names += all_td_str

In [ ]:
with open('./chinese_names.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(names)

In [ ]:
for year in range(1880, 1930):
    post_params = {
    'top' : '100',
    'year' : str(year)
    }
    post_args = urllib.urlencode(post_params)
    urllib2.urlopen(url, post_args)
    headers = {
        'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language' : 'en-US,en;q=0.5',
        'Connection' : 'keep-alive',
        'Host' : 'www.ssa.gov',
        'Referer' : 'http://www.ssa.gov/cgi-bin/popularnames.cgi',
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) Gecko/20100101 Firefox/21.0'
        }

    # With POST data:
    page = urllib.urlopen(url, post_args, headers)
    soup = BeautifulSoup(page.read(), 'lxml')
    rank = soup.find('table', {'border': '1', 'bordercolor' : '#aaaabb'})
    all_td = rank.find_all('td')
    all_td_str = [x.string for x in all_td]
    all_td_str.pop(-1)
    B = np.reshape(all_td_str, (-1, 3))
    B = [list(x)[1:] for x in B]
    
    with open('./names/'+str(year)+'_names.csv', 'w') as f:
        w = csv.writer(f)
        w.writerows(B)